In [41]:
import requests
import json

def extract_initials(name):
    name_parts = name.split()
    first_initial = name_parts[0][0] + "."
    middle_initial = ""
    if len(name_parts) > 1:
        middle_initial = name_parts[1][0] + "."
    last_name = name_parts[-1]
    return first_initial, middle_initial, last_name

def format_author_name(author):
    """
    Formats an author's name in APA 7 style.

    Args:
        author (dict): The author's information containing given and family names.

    Returns:
        str: The formatted author name.
    """
    given_name = author.get("given", "")
    family_name = author.get("family", "")
    if given_name and family_name:
        first_initial, middle_initial, last_name = extract_initials(given_name)  # Abbreviate given name
        formatted_name = f"{family_name}, {first_initial} {middle_initial}"
    else:
        formatted_name = f"{given_name} {family_name}"
    return formatted_name


def generate_citation_apa7(authors, year, title, source, doi):
    """
    Generates a citation in APA 7 style.

    Args:
        authors (str or list): The author(s) of the source.
        year (int): The year the source was published.
        title (str): The title of the source.
        source (str): The type of source (e.g., journal article, book, website).

    Returns:
        str: The formatted APA 7 citation.
    """
    # Format authors
    if isinstance(authors, str):
        formatted_authors = authors
    elif isinstance(authors, list):
        formatted_authors = ", ".join([format_author_name(author) for author in authors])
    else:
        raise ValueError("Invalid 'authors' argument. Expected string or list.")

    # Format year
    formatted_year = str(year)

    # Format title
    formatted_title = f'"{title}"'

    # Format source
    formatted_source = source.capitalize()
    
    # Format DOI
    formatted_doi = f"DOI: {doi}"

    # Combine elements into citation
    citation = f'{formatted_authors} ({formatted_year}). {formatted_title}. {formatted_source}. {formatted_doi}'

    return citation


def generate_citation_from_doi(doi):
    """
    Generates a citation in APA 7 style from a DOI.

    Args:
        doi (str): The DOI (Digital Object Identifier) of the source.

    Returns:
        str: The formatted APA 7 citation.
    """
    # Fetch metadata from CrossRef API
    response = requests.get(f"https://api.crossref.org/works/{doi}")
    if response.status_code == 200:
        data = json.loads(response.text)
        metadata = data["message"]
    else:
        return "Error: Unable to retrieve metadata for the given DOI."

    # Extract relevant metadata
    authors = []
    if "author" in metadata:
        for author in metadata["author"]:
            authors.append(author)
    year = metadata.get("created", {}).get("date-parts", [[]])[0][0]
    title = metadata.get("title", [])[0]
    source = metadata.get("container-title", [""])[0]
    doi = metadata.get("DOI", "")
    
    # Generate citation
    return generate_citation_apa7(authors, year, title, source, doi)

doi = input("Please paste the DOI:")
citation = generate_citation_from_doi(doi)
print(citation)

Please paste the DOI: https://doi.org/10.1021/acs.jctc.2c01303


Horne, R. I., Murtada, M. H., Huo, D. , Brotzakis, Z. F., Gregory, R. C., Possenti, A. , Chia, S. , Vendruscolo, M.  (2023). "Exploration and Exploitation Approaches Based on Generative Machine Learning to Identify Potent Small Molecule Inhibitors of α-Synuclein Secondary Nucleation". Journal of chemical theory and computation. DOI: 10.1021/acs.jctc.2c01303
